In [1]:
%load_ext autoreload
%autoreload 2

In [21]:
import os
import yaml
import pandas as pd
from IPython.display import display
import src.util as ut
from src.util import esc_ac, esc_sc, esc_ec
import matplotlib.pyplot as plt
import shutil
import numpy as np
import csv
res_dir = 'res_agg/' # 'res/'

### Result Table (LPLM dataset)

In [3]:
res_dir = 'res_agg_bak3/' # 'res/'
# data_names = ['WIKI', 'IMDB', 'DBLP', 'GENE', 'AUTHOR', 'DBLP-AN', 'IMDb-AN', 'IMDb-MT', 'TPCH-PN']
data_names = ['DBLP-AN', 'IMDb-AN', 'IMDb-MT', 'TPCH-PN']
models = ['EST_S', 'EST_M', 'EST_B', 'E2E', 'E2E-AUG', 'Astrid', 'Astrid-AUG', 'DREAM', 'DREAM-PACK', 'CLIQUE', 'CLIQUE-PACK', 'CLIQUE-PACK-ALL', 'LPLM']
# models = ['CLIQUE', 'CLIQUE-PACK', 'CLIQUE-PACK-ALL']
# workloads = ['CLIQUE', 'LPLM']
# workloads = ['LPLM', 'LPLM1']
workloads = ['LPLM']
measures = ['avg', 'q050', 'q090', 'q099', 'q100']


for workload in workloads:
    res_dict = {}
    for data_name in data_names:
        for model in models:
            #### Get only one stat ####
            # stat_path = os.path.join(res_dir, data_name, workload, 'stat', model, '0', 'stat.yml')
            # # print(stat_path)
            # if os.path.exists(stat_path):
            #     stat_dict = yaml.load(open(stat_path), Loader=yaml.FullLoader)
            #     stat_func = lambda x: stat_dict[x]
            # else:
            #     stat_func = lambda x: None

            #### Get average stat ####
            stat_dicts = []
            for trial in [0, 1, 2]:
                stat_path = os.path.join(res_dir, data_name, workload, 'stat', model, '0', 'stat.yml')
                # print(stat_path)
                if os.path.exists(stat_path):
                    stat_dict = yaml.load(open(stat_path), Loader=yaml.FullLoader)
                    stat_dicts.append(stat_dict)
            if len(stat_dicts) > 0:
                stat_func = lambda x: np.average([stat_dict[x] for stat_dict in stat_dicts])
            else:
                stat_func = lambda x: None

            #### make res_dict
            for measure in measures:
                if (data_name, measure) not in res_dict:
                    res_dict[(data_name, measure)] = []
                res_dict[(data_name, measure)].append(stat_func(measure))
    df = pd.DataFrame(res_dict, index=models)
    df = df.style.format(na_rep='', precision=2)
    # print(f"{workload =}")
    display(df)


### Result Table (LPLM dataset CLIQUE fixed)

In [5]:
db, train_data, valid_data, test_data = ut.load_training_files(
    "DBLP-AN", "LPLM")

d_type = train     training_path = 'data/DBLP-AN/training/LPLM/train.txt'
d_type = valid     training_path = 'data/DBLP-AN/training/LPLM/valid.txt'
d_type = test      training_path = 'data/DBLP-AN/training/LPLM/test.txt'


In [25]:
res_dir = 'res_agg_bak3_fixed/' # 'res/'
data_names = ['DBLP-AN', 'IMDb-AN', 'IMDb-MT', 'TPCH-PN']
models = ['CLIQUE', 'CLIQUE-PACK', 'CLIQUE-PACK-ALL']
# data_name = "DBLP-AN"
# model = "CLIQUE"
for data_name in data_names:
    for model in models:
        ME = yaml.load(open(f"{res_dir}{data_name}/LPLM/model/CLIQUE/0/config.yml"), Loader=yaml.FullLoader)["max_entry_ratio"]
        N = yaml.load(open(f"{res_dir}{data_name}/LPLM/model/CLIQUE/0/config.yml"), Loader=yaml.FullLoader)["N"]
        print(ME, N)
        hashtable, PTs, _ = ut.getCachedPrunedExtendedNgramTableMaxEntry(data_name, N, ME)
        estimation_path = f"{res_dir}{data_name}/LPLM/estimation/{model}/0/estimation.csv"
        stat_path = f"{res_dir}{data_name}/LPLM/stat/{model}/0/stat.yml"
        print(f"{estimation_path = }")
        print(f"{stat_path = }")

        csv_r = csv.reader(open(estimation_path))
        new_lines = []
        header = next(csv_r)
        new_lines.append(header)
        q_errs=[]
        for line in csv_r:
            query, true_card, est_card, q_err = line
            true_card = int(true_card)
            est_card = float(est_card)
            q_err = float(q_err)
            test_query_norm = ut.flip_last_canonicalized_like_query(query)
            # print(line)
            ngram = test_query_norm.strip("%").replace("_", esc_ac)
            is_prefix_query = test_query_norm[0] != "%"
            is_suffix_query = test_query_norm[-1] != "%"
            if is_prefix_query:
                ngram = esc_sc + ngram
            if is_suffix_query:
                ngram = ngram + esc_ec
            if ngram in hashtable:
                est_card = hashtable[ngram]
                q_err = ut.mean_Q_error(true_card, est_card)
                assert int(q_err) == 1
                line = query, true_card, est_card, q_err
            new_lines.append(line)
            q_errs.append(q_err)
        # print(new_lines)
        ut.save_estimator_stat(stat_path, q_errs, 0,
                            0, model_size=0)

3.15 5
time_path = 'res/DBLP-AN/Ntable/count_N5_ME3.15.yml'
estimation_path = 'res_agg_bak3_fixed/DBLP-AN/LPLM/estimation/CLIQUE/0/estimation.csv'
stat_path = 'res_agg_bak3_fixed/DBLP-AN/LPLM/stat/CLIQUE/0/stat.yml'
out_stat_dict['avg'] = 2.265742226731955, out_stat_dict['q100'] = 239.5
stat_path = 'res_agg_bak3_fixed/DBLP-AN/LPLM/stat/CLIQUE/0/stat.yml'
3.15 5
time_path = 'res/DBLP-AN/Ntable/count_N5_ME3.15.yml'
estimation_path = 'res_agg_bak3_fixed/DBLP-AN/LPLM/estimation/CLIQUE-PACK/0/estimation.csv'
stat_path = 'res_agg_bak3_fixed/DBLP-AN/LPLM/stat/CLIQUE-PACK/0/stat.yml'
out_stat_dict['avg'] = 3.0605296471916947, out_stat_dict['q100'] = 954.255126953125
stat_path = 'res_agg_bak3_fixed/DBLP-AN/LPLM/stat/CLIQUE-PACK/0/stat.yml'
3.15 5
time_path = 'res/DBLP-AN/Ntable/count_N5_ME3.15.yml'
estimation_path = 'res_agg_bak3_fixed/DBLP-AN/LPLM/estimation/CLIQUE-PACK-ALL/0/estimation.csv'
stat_path = 'res_agg_bak3_fixed/DBLP-AN/LPLM/stat/CLIQUE-PACK-ALL/0/stat.yml'
out_stat_dict['avg'] = 3.

In [27]:
res_dir = 'res_agg_bak3_fixed/' # 'res/'
# data_names = ['WIKI', 'IMDB', 'DBLP', 'GENE', 'AUTHOR', 'DBLP-AN', 'IMDb-AN', 'IMDb-MT', 'TPCH-PN']
data_names = ['DBLP-AN', 'IMDb-AN', 'IMDb-MT', 'TPCH-PN']
models = ['EST_S', 'EST_M', 'EST_B', 'E2E', 'E2E-AUG', 'Astrid', 'Astrid-AUG', 'DREAM', 'DREAM-PACK', 'CLIQUE', 'CLIQUE-PACK', 'CLIQUE-PACK-ALL', 'LPLM']
models = ['DREAM', 'DREAM-PACK', 'CLIQUE', 'CLIQUE-PACK', 'CLIQUE-PACK-ALL', 'LPLM']
# models = ['CLIQUE', 'CLIQUE-PACK', 'CLIQUE-PACK-ALL']
# workloads = ['CLIQUE', 'LPLM']
# workloads = ['LPLM', 'LPLM1']
workloads = ['LPLM']
measures = ['avg', 'q050', 'q090', 'q099', 'q100']


for workload in workloads:
    res_dict = {}
    for data_name in data_names:
        for model in models:
            #### Get only one stat ####
            # stat_path = os.path.join(res_dir, data_name, workload, 'stat', model, '0', 'stat.yml')
            # # print(stat_path)
            # if os.path.exists(stat_path):
            #     stat_dict = yaml.load(open(stat_path), Loader=yaml.FullLoader)
            #     stat_func = lambda x: stat_dict[x]
            # else:
            #     stat_func = lambda x: None

            #### Get average stat ####
            stat_dicts = []
            for trial in [0, 1, 2]:
                stat_path = os.path.join(res_dir, data_name, workload, 'stat', model, '0', 'stat.yml')
                # print(stat_path)
                if os.path.exists(stat_path):
                    stat_dict = yaml.load(open(stat_path), Loader=yaml.FullLoader)
                    stat_dicts.append(stat_dict)
            if len(stat_dicts) > 0:
                stat_func = lambda x: np.average([stat_dict[x] for stat_dict in stat_dicts])
            else:
                stat_func = lambda x: None

            #### make res_dict
            for measure in measures:
                if (data_name, measure) not in res_dict:
                    res_dict[(data_name, measure)] = []
                res_dict[(data_name, measure)].append(stat_func(measure))
    df = pd.DataFrame(res_dict, index=models)
    df = df.style.format(na_rep='', precision=2)
    # print(f"{workload =}")
    display(df)


### Result Table (CLIQUE dataset)

In [23]:
data_names = ['WIKI', 'IMDB', 'DBLP', 'GENE', 'AUTHOR']
# data_names = ['DBLP-AN', 'IMDb-AN', 'IMDb-MT', 'TPCH-PN']
models = ['EST_S', 'EST_M', 'EST_B', 'E2E', 'E2E-AUG', 'Astrid', 'Astrid-AUG', 'DREAM', 'DREAM-PACK', 'CLIQUE', 'CLIQUE-PACK', 'CLIQUE-PACK-ALL', 'LPLM']
# models = ['CLIQUE', 'CLIQUE-PACK', 'CLIQUE-PACK-ALL']
# workloads = ['CLIQUE', 'LPLM']
# workloads = ['LPLM', 'LPLM1']
workloads = ['CLIQUE']
measures = ['avg', 'q050', 'q090', 'q099', 'q100']


for workload in workloads:
    res_dict = {}
    for data_name in data_names:
        for model in models:
            #### Get only one stat ####
            # stat_path = os.path.join(res_dir, data_name, workload, 'stat', model, '0', 'stat.yml')
            # # print(stat_path)
            # if os.path.exists(stat_path):
            #     stat_dict = yaml.load(open(stat_path), Loader=yaml.FullLoader)
            #     stat_func = lambda x: stat_dict[x]
            # else:
            #     stat_func = lambda x: None

            #### Get average stat ####
            stat_dicts = []
            for trial in [0, 1, 2]:
                stat_path = os.path.join(res_dir, data_name, workload, 'stat', model, '0', 'stat.yml')
                # print(stat_path)
                if os.path.exists(stat_path):
                    stat_dict = yaml.load(open(stat_path), Loader=yaml.FullLoader)
                    stat_dicts.append(stat_dict)
            if len(stat_dicts) > 0:
                stat_func = lambda x: np.average([stat_dict[x] for stat_dict in stat_dicts])
            else:
                stat_func = lambda x: None
            
            #### make res_dict
            for measure in measures:
                if (data_name, measure) not in res_dict:
                    res_dict[(data_name, measure)] = []
                res_dict[(data_name, measure)].append(stat_func(measure))
    df = pd.DataFrame(res_dict, index=models)
    df = df.style.format(na_rep='', precision=2)
    # print(f"{workload =}")
    display(df)


### Result Table (Astrid)

In [39]:
res_dir = "res_agg/"
data_names = ['WIKI', 'IMDB', 'DBLP', 'GENE', 'AUTHOR']
models = ['EST_S', 'EST_M', 'EST_B', 'E2E', 'E2E-AUG', 'Astrid', 'Astrid-AUG', 'DREAM', 'DREAM-PACK', 'CLIQUE', 'CLIQUE-PACK-ALL', 'LPLM', 'AstridEach']
# models = ['CLIQUE', 'CLIQUE-PACK-ALL', 'LPLM', 'AstridEach']
# models = ['AstridEach']
workloads = ['Astrid']
measures = ['avg', 'q050', 'q090', 'q099', 'q100']


for workload in workloads:
    res_dict = {}
    for data_name in data_names:
        for model in models:
            #### Get only one stat ####
            # stat_path = os.path.join(res_dir, data_name, workload, 'stat', model, '0', 'stat.yml')
            # # print(stat_path)
            # if os.path.exists(stat_path):
            #     stat_dict = yaml.load(open(stat_path), Loader=yaml.FullLoader)
            #     stat_func = lambda x: stat_dict[x]
            # else:
            #     stat_func = lambda x: None

            #### Get average stat ####
            stat_dicts = []
            for trial in [0, 1, 2]:
                stat_path = os.path.join(res_dir, data_name, workload, 'stat', model, '0', 'stat.yml')
                # print(stat_path)
                if os.path.exists(stat_path):
                    stat_dict = yaml.load(open(stat_path), Loader=yaml.FullLoader)
                    stat_dicts.append(stat_dict)
            if len(stat_dicts) > 0:
                stat_func = lambda x: np.average([stat_dict[x] for stat_dict in stat_dicts])
            else:
                stat_func = lambda x: None

            #### make res_dict
            for measure in measures:
                if (data_name, measure) not in res_dict:
                    res_dict[(data_name, measure)] = []
                res_dict[(data_name, measure)].append(stat_func(measure))
    df = pd.DataFrame(res_dict, index=models)
    df = df.style.format(na_rep='', precision=2)
    # print(f"{workload =}")
    display(df)


### Result Table (LPLM20)

In [38]:
res_dir = "res_agg/"
data_names = ['WIKI', 'IMDB', 'DBLP', 'GENE', 'AUTHOR']
models = ['EST_S', 'EST_M', 'EST_B', 'E2E', 'E2E-AUG', 'Astrid', 'Astrid-AUG', 'DREAM', 'DREAM-PACK', 'CLIQUE', 'CLIQUE-PACK-ALL', 'LPLM', 'AstridEach']
# models = ['CLIQUE', 'CLIQUE-PACK-ALL', 'LPLM', 'AstridEach']
# models = ['AstridEach']
workloads = ['LPLM20']
measures = ['avg', 'q050', 'q090', 'q099', 'q100']


for workload in workloads:
    res_dict = {}
    for data_name in data_names:
        for model in models:
            #### Get only one stat ####
            # stat_path = os.path.join(res_dir, data_name, workload, 'stat', model, '0', 'stat.yml')
            # # print(stat_path)
            # if os.path.exists(stat_path):
            #     stat_dict = yaml.load(open(stat_path), Loader=yaml.FullLoader)
            #     stat_func = lambda x: stat_dict[x]
            # else:
            #     stat_func = lambda x: None

            #### Get average stat ####
            stat_dicts = []
            for trial in [0, 1, 2]:
                stat_path = os.path.join(res_dir, data_name, workload, 'stat', model, '0', 'stat.yml')
                # print(stat_path)
                if os.path.exists(stat_path):
                    stat_dict = yaml.load(open(stat_path), Loader=yaml.FullLoader)
                    stat_dicts.append(stat_dict)
            if len(stat_dicts) > 0:
                stat_func = lambda x: np.average([stat_dict[x] for stat_dict in stat_dicts])
            else:
                stat_func = lambda x: None

            #### make res_dict
            for measure in measures:
                if (data_name, measure) not in res_dict:
                    res_dict[(data_name, measure)] = []
                res_dict[(data_name, measure)].append(stat_func(measure))
    df = pd.DataFrame(res_dict, index=models)
    df = df.style.format(na_rep='', precision=2)
    # print(f"{workload =}")
    display(df)


### Move the submit result to the revision result

#### Get info

In [18]:
infos = [  # model, query_key, ename
    ("PSQL", "ru100/2_2", "PostgreSQL"),
    ("LBS/L10", "ru100/2_2", "LBS"),
    ("Fixed/1000/", "ru100/2_2", "EST_S"),
    ("GreekFixed/1000/", "ru100/2_2", "EST_M"),
    (
        {
            "WIKI": "GreekAdapt/100.0/",
            "IMDB": "GreekAdapt/100.0/",
            "DBLP": "GreekAdapt/20.0/",
            "GENE": "GreekAdapt/10.0/",
            "AUTHOR": "GreekAdapt/50.0/",
        },
        "ru100/2_2",
        "EST_B",
    ),
    ("Astrid/lr0.001/bs128/ep256/pool/ms5/ed1024/es8/pt1.0/pat20", "ru100/2_2", "Astrid"),
    ("E2E", "ru100/2_2", "E2E"),
    ("LSTM/bs64/p20/pt1.0/bs64/pt1.0/phs256/rhs128/ces500/npl5", "ru100/2_2", "DREAM"),
    (
        {
            "WIKI": "hybrid_logbound_bn/N5/ME4.35/ns0/bs64/pt1.0/phs128/rhs256/ces100/npl3",
            "IMDB": "hybrid_logbound_bn/N5/ME1.675/ns0/bs64/pt1.0/phs128/rhs256/ces100/npl3",
            "DBLP": "hybrid_logbound_bn/N5/ME3.12/ns0/bs64/pt1.0/phs128/rhs256/ces100/npl3",
            "GENE": "hybrid_logbound_bn/N5/ME204.8/ns0/bs64/pt1.0/phs128/rhs256/ces100/npl3",
            "AUTHOR": "hybrid_logbound_bn/N5/ME4.8/ns0/bs64/pt1.0/phs128/rhs256/ces100/npl3",
        },
        "ru100/2_2",
        "CLIQUE",
    ),
    ("Astrid_packed/lr0.001/bs128/ep256/pool/ms5/ed1024/es8/pt1.0/pat20", "ru100/2_2", "Astrid-AUG"),
    ("E2E_packed/pt1.0", "ru100/2_2", "E2E-AUG"),
    ("simple/LSTM/bs64/p20/pt1.0/bs64/pt1.0/phs256/rhs128/ces500/npl5", "ru100/2_2", "DREAM-PACK"),
    # ("LPLM_packed/pt1.0", "ru100/2_2", "LPLM-PACK"),
    (
        {
            "WIKI": "LPLM_packed/pt1.0/h780",
            "IMDB": "LPLM_packed/pt1.0/h390",
            "DBLP": "LPLM_packed/pt1.0/h392",
            "GENE": "LPLM_packed/pt1.0/h410",
            "AUTHOR": "LPLM_packed/pt1.0/h396",
        }, 
        "ru100/2_2",
        "LPLM"
    ),
    (
        {
            "WIKI": "v1/hybrid_packed_logbound_bn/N5/ME4.35/ns0/bs64/pt1.0/phs128/rhs256/ces100/npl3",
            "IMDB": "v1/hybrid_packed_logbound_bn/N5/ME1.675/ns0/bs64/pt1.0/phs128/rhs256/ces100/npl3",
            "DBLP": "v1/hybrid_packed_logbound_bn/N5/ME3.12/ns0/bs64/pt1.0/phs128/rhs256/ces100/npl3",
            "GENE": "v1/hybrid_packed_logbound_bn/N5/ME204.8/ns0/bs64/pt1.0/phs128/rhs256/ces100/npl3",
            "AUTHOR": "v1/hybrid_packed_logbound_bn/N5/ME4.8/ns0/bs64/pt1.0/phs128/rhs256/ces100/npl3",
        },
        "ru100/2_2",
        "CLIQUE-PACK",
    ),
]
info_dict = {
    z: (x, y) for x, y, z in infos
}

#### move file (stat only)

In [20]:
submit_dir = 'res_submit/res/'

data_names = ['WIKI', 'IMDB', 'DBLP', 'GENE', 'AUTHOR']
# data_names = ['DBLP-AN', 'IMDb-AN', 'IMDb-MT', 'TPCH-PN']
models = ['EST_S', 'EST_M', 'EST_B', 'E2E', 'E2E-AUG', 'Astrid', 'Astrid-AUG', 'DREAM', 'DREAM-PACK', 'CLIQUE', 'CLIQUE-PACK', 'LPLM']
model_dict = {
    'CLIQUE-PACK': "",
}
# models = ['CLIQUE', 'CLIQUE-PACK', 'CLIQUE-PACK-ALL']
workload = 'CLIQUE'
# measures = ['avg', 'q050', 'q090', 'q099', 'q100']

for data_name in data_names:
    for model in models:
        exp_info_or_exp_info_dict, q_type = info_dict[model] # ru/100
        if isinstance(exp_info_or_exp_info_dict, dict):
            exp_info = exp_info_or_exp_info_dict[data_name]
        else:
            exp_info = exp_info_or_exp_info_dict

        # stat path 
        for trial in [0, 1, 2]:
            src_path = os.path.join(submit_dir, data_name, str(trial+1), 'stat', exp_info, q_type + '.yml')
            dest_path = os.path.join(res_dir, data_name, workload, 'stat', model, str(trial), 'stat.yml')
            # if os.path.exists(dest_path):
            #     print(f"{dest_path = }")
            
            if not os.path.exists(src_path):
                print(f"Not exists {src_path= }")
                continue
            if not os.path.exists(dest_path):
                print(f"Copy from {src_path = } to {dest_path = }")
                os.makedirs(os.path.dirname(dest_path), exist_ok=True)
                shutil.copy(src_path, dest_path)

        # if not os.path.exists(src_path):
        #     print(f"{src_path = }")

    


Not exists src_path= 'res_submit/res/WIKI/2/stat/Astrid_packed/lr0.001/bs128/ep256/pool/ms5/ed1024/es8/pt1.0/pat20/ru100/2_2.yml'
Not exists src_path= 'res_submit/res/IMDB/2/stat/Astrid_packed/lr0.001/bs128/ep256/pool/ms5/ed1024/es8/pt1.0/pat20/ru100/2_2.yml'


In [19]:
df = pd.read_csv("res_agg/TPCH-PN/LPLM1/estimation/CLIQUE-PACK/0/estimation.csv")

In [20]:
df

,query,true,est,q-error
0,% % % %,200000,186417.420000,1.072861
1,% % % % %O%,78563,59881.445000,1.311976
2,% % % %A%E%E_ANTIQ%,23,30.610909,1.330909
3,% % % %C%,64117,54845.940000,1.169038
4,% % % %CH%D_D%R,148,117.295080,1.261775
...,...,...,...,...
499995,_Z_R_ _I%E%D_,5,2.431440,2.056394
499996,_Z_R_ _I_T%SN_W%ID%L,1,0.008862,1.000000
499997,_Z_R_ _I_TY%L_ON%D_E_ C%L_WE_,1,0.968121,1.000000
499998,_Z_R_ _OWDER%S%RQ%I% SM%E,1,0.889470,1.000000


### Analysis Result

In [18]:
def count_wild(query):
    n_under = query.count('_')
    n_pcnt = query.count('%')
    return n_under + n_pcnt


# data_names = ['DBLP-AN', 'IMDb-AN', 'IMDb-MT', 'TPCH-PN']
# data_names = ['IMDb-MT']
data_names = ['DBLP-AN']
# data_names = ['IMDb-MT']
# models = ['EST_S', 'EST_M', 'EST_B', 'E2E', 'E2E-AUG', 'Astrid', 'Astrid-AUG', 'DREAM', 'DREAM-PACK', 'CLIQUE', 'CLIQUE-PACK', 'LPLM']
# models = ['DREAM', 'DREAM-PACK', 'CLIQUE', 'CLIQUE-PACK']
# models = ['LPLM','CLIQUE', 'CLIQUE-PACK', 'CLIQUE-PACK-ALL']
# models = ['LPLM','CLIQUE', 'CLIQUE-PACK']
models = ['CLIQUE', 'CLIQUE-PACK']
# workloads = ['CLIQUE', 'LPLM']
workloads = ['LPLM', 'LPLM1']


for workload in workloads:
    res_dict = {}
    for data_name in data_names:
        for model in models:
            # stat_path = os.path.join(res_dir, data_name, workload, 'stat', model, '0', 'stat.yml')
            stat_path = os.path.join(res_dir, data_name, workload, 'estimation', model, '0', 'estimation.csv')
            if not os.path.exists(stat_path):
                continue
            print(f"{workload = } {data_name = } {model = } {stat_path = }")
            df = pd.read_csv(stat_path)
            df['n_wild'] = df['query'].apply(count_wild)
            print(model)
            print(f"{df['n_wild'].mean() = }")
            df['len'] = df['query'].apply(len)
            # df['p'] = df['query'].apply(lambda x: 1)
            # df = df.groupby('n_wild').mean('q-error').transpose()
            df = df.groupby('len').mean('q-error').transpose()
            df = df.style.format(na_rep='', precision=2)
            display(df)
            # display(df)

workload = 'LPLM' data_name = 'DBLP-AN' model = 'CLIQUE' stat_path = 'res_agg/DBLP-AN/LPLM/estimation/CLIQUE/0/estimation.csv'
CLIQUE
df['n_wild'].mean() = 3.256418


len,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41
true,25571.57,1979.18,1932.30,1773.02,1071.68,961.50,371.55,208.74,36.85,16.31,8.19,6.77,6.19,5.82,5.87,5.88,5.74,5.80,5.39,4.83,5.23,5.02,4.50,4.83,5.78,3.72,3.16,4.21,3.68,2.99,3.66,5.71,3.29,2.50,3.75,4.43,2.75,3.33,1.00,1.00
est,25571.51,2030.69,2089.12,1658.47,974.42,856.38,327.63,185.38,30.55,12.72,6.21,5.20,4.86,4.60,4.80,4.85,4.81,4.93,4.67,3.99,4.54,4.36,3.87,4.39,5.21,3.16,2.62,3.91,3.19,3.20,4.14,4.66,2.16,1.68,2.68,2.32,1.68,1.60,1.97,1.73
q-error,1.00,1.52,1.94,1.84,1.80,1.89,2.09,2.32,2.44,2.44,2.40,2.34,2.30,2.24,2.22,2.17,2.13,2.10,2.05,2.04,2.02,1.98,1.97,1.93,1.95,1.94,1.95,1.84,1.87,1.80,1.87,2.02,1.88,1.78,1.66,2.11,2.26,2.58,1.97,1.73
n_wild,1.00,1.03,1.52,1.99,2.59,3.00,3.32,3.49,3.30,3.22,3.18,3.18,3.20,3.23,3.29,3.38,3.47,3.56,3.66,3.71,3.80,3.88,3.92,3.93,3.99,4.28,4.32,4.34,4.19,4.48,4.53,4.60,4.36,3.59,4.12,4.43,7.50,7.00,7.00,3.00


workload = 'LPLM' data_name = 'DBLP-AN' model = 'CLIQUE-PACK' stat_path = 'res_agg/DBLP-AN/LPLM/estimation/CLIQUE-PACK/0/estimation.csv'
CLIQUE-PACK
df['n_wild'].mean() = 3.256418


len,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41
true,25571.57,1979.18,1932.30,1773.02,1071.68,961.50,371.55,208.74,36.85,16.31,8.19,6.77,6.19,5.82,5.87,5.88,5.74,5.80,5.39,4.83,5.23,5.02,4.50,4.83,5.78,3.72,3.16,4.21,3.68,2.99,3.66,5.71,3.29,2.50,3.75,4.43,2.75,3.33,1.00,1.00
est,9284.86,1312.09,1874.59,1737.57,1036.53,939.04,358.16,202.08,32.22,12.04,4.67,3.55,3.17,3.11,3.38,3.57,3.61,3.72,3.45,2.85,3.33,3.34,3.06,3.69,5.76,2.31,2.08,2.38,2.08,1.95,2.13,2.75,1.21,2.60,2.53,2.37,2.71,4.69,2.63,2.45
q-error,3.92,1.48,1.49,1.66,1.81,2.10,2.45,3.00,3.37,3.45,3.40,3.31,3.21,3.09,3.01,2.94,2.87,2.88,2.80,2.69,2.65,2.58,2.54,2.37,2.36,2.40,2.36,2.61,2.68,2.11,2.44,3.20,2.58,2.22,2.25,3.02,1.81,3.81,2.63,2.45
n_wild,1.00,1.03,1.52,1.99,2.59,3.00,3.32,3.49,3.30,3.22,3.18,3.18,3.20,3.23,3.29,3.38,3.47,3.56,3.66,3.71,3.80,3.88,3.92,3.93,3.99,4.28,4.32,4.34,4.19,4.48,4.53,4.60,4.36,3.59,4.12,4.43,7.50,7.00,7.00,3.00


workload = 'LPLM1' data_name = 'DBLP-AN' model = 'CLIQUE' stat_path = 'res_agg/DBLP-AN/LPLM1/estimation/CLIQUE/0/estimation.csv'
CLIQUE
df['n_wild'].mean() = 3.256418


len,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41
true,25571.57,1979.18,1932.30,1773.02,1071.68,961.50,371.55,208.74,36.85,16.31,8.19,6.77,6.19,5.82,5.87,5.88,5.74,5.80,5.39,4.83,5.23,5.02,4.50,4.83,5.78,3.72,3.16,4.21,3.68,2.99,3.66,5.71,3.29,2.50,3.75,4.43,2.75,3.33,1.00,1.00
est,25046.02,2055.12,2139.53,1797.03,1038.53,910.24,346.08,193.40,31.91,13.12,6.20,5.12,4.73,4.43,4.59,4.74,4.68,4.71,4.49,3.92,4.40,4.18,3.85,4.17,5.05,2.91,2.75,3.42,2.62,2.66,3.13,4.74,1.95,1.61,2.00,1.92,1.56,1.87,1.77,1.74
q-error,1.21,1.56,1.88,1.83,1.80,1.88,2.08,2.33,2.42,2.41,2.37,2.32,2.26,2.23,2.18,2.13,2.10,2.07,2.04,2.03,2.01,1.92,1.96,1.87,1.89,1.94,1.89,1.88,1.88,1.74,1.74,2.00,1.79,1.78,2.09,2.59,2.13,2.22,1.77,1.74
n_wild,1.00,1.03,1.52,1.99,2.59,3.00,3.32,3.49,3.30,3.22,3.18,3.18,3.20,3.23,3.29,3.38,3.47,3.56,3.66,3.71,3.80,3.88,3.92,3.93,3.99,4.28,4.32,4.34,4.19,4.48,4.53,4.60,4.36,3.59,4.12,4.43,7.50,7.00,7.00,3.00


workload = 'LPLM1' data_name = 'DBLP-AN' model = 'CLIQUE-PACK' stat_path = 'res_agg/DBLP-AN/LPLM1/estimation/CLIQUE-PACK/0/estimation.csv'
CLIQUE-PACK
df['n_wild'].mean() = 3.256418


len,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41
true,25571.57,1979.18,1932.30,1773.02,1071.68,961.50,371.55,208.74,36.85,16.31,8.19,6.77,6.19,5.82,5.87,5.88,5.74,5.80,5.39,4.83,5.23,5.02,4.50,4.83,5.78,3.72,3.16,4.21,3.68,2.99,3.66,5.71,3.29,2.50,3.75,4.43,2.75,3.33,1.00,1.00
est,9624.27,1048.73,1756.47,1640.15,1006.25,909.84,343.28,193.56,30.14,11.47,4.51,3.50,3.29,3.23,3.57,3.90,3.87,3.87,3.79,3.17,3.98,3.60,3.49,3.91,5.74,2.76,2.32,2.75,2.39,2.02,2.61,3.58,1.31,2.54,2.47,1.41,2.66,5.22,3.83,3.70
q-error,2.73,1.60,1.59,1.69,1.80,2.11,2.47,3.01,3.38,3.46,3.45,3.33,3.23,3.09,3.01,2.94,2.85,2.87,2.78,2.65,2.62,2.58,2.50,2.40,2.25,2.42,2.52,2.45,2.63,2.11,2.39,2.71,2.62,1.94,2.91,3.75,2.00,2.69,3.83,3.70
n_wild,1.00,1.03,1.52,1.99,2.59,3.00,3.32,3.49,3.30,3.22,3.18,3.18,3.20,3.23,3.29,3.38,3.47,3.56,3.66,3.71,3.80,3.88,3.92,3.93,3.99,4.28,4.32,4.34,4.19,4.48,4.53,4.60,4.36,3.59,4.12,4.43,7.50,7.00,7.00,3.00


In [27]:
def count_wild(query):
    n_under = query.count('_')
    n_pcnt = query.count('%')
    return n_under + n_pcnt

df['n_wild'] = df['query'].apply(count_wild)

In [38]:
df.groupby('n_wild').mean('q-error')

,true,est,q-error
n_wild,,,
0,1.675374,1.562978,1.186668
1,9.856057,9.183350,1.440785
2,78.201509,81.339569,1.500338
3,253.574261,260.642012,1.535603
4,508.126504,537.736610,1.509873
5,454.968743,511.864816,1.460435
6,114.916975,130.276895,1.381826
7,27.228493,31.723621,1.290050
8,5.415208,5.969275,1.202779


In [65]:
import pickle
with open("res/IMDb-MT/Ntable/count_N5_ME1.6.pkl", 'rb') as f:
    table = pickle.load(f)

In [66]:
type(table)
table[1]

[2, 2, 2, 562, 2246, 4004]

In [69]:
df = pd.read_csv("data/DBLP/training/Astrid/train.txt")

In [72]:
df.values[0][0] == df.values[1][0]

True

In [67]:
query = '_6'
ut.esc_sc # start
ut.esc_ac # any
ut.esc_ec # end
n_gram = ut.esc_sc + query.replace('_', ut.esc_ac) + ut.esc_ec
print(f"{n_gram = }")
print(f"{table[0][n_gram] = }")

n_gram = '˂‼6˃'
table[0][n_gram] = 2481


In [ ]:
table['']

In [56]:
list(table)[:10]

[{'': 500000,
  '˂I': 10527,
  '˂‼': 500000,
  'S˃': 51395,
  '‼˃': 500000,
  ' ': 308347,
  'W': 61424,
  'S': 218525,
  '‼': 500000,
  'A': 267677,
  'F': 84093,
  'I': 238810,
  'O': 228737,
  'B': 85524,
  'N': 232242,
  'R': 234970,
  'T': 233542,
  'H': 165030,
  'V': 60451,
  'E': 290809,
  '˂P': 16103,
  'E˃': 54200,
  'P': 94677,
  'C': 129639,
  '˂S': 29163,
  'M': 132333,
  'K': 74027,
  'N˃': 33329,
  'U': 135510,
  '˂T': 54900,
  'W˃': 3255,
  'L': 180701,
  '˂A': 25178,
  'P˃': 3861,
  'Z': 20799,
  'R˃': 24316,
  '˂R': 11320,
  'D': 157788,
  '˂H': 13651,
  '˂L': 21364,
  'G': 110441,
  'Y': 80187,
  '˂M': 20030,
  '˂7': 1479,
  '0˃': 13483,
  '7': 33021,
  '1': 125585,
  '0': 78761,
  'A˃': 24884,
  '˂1': 72031,
  '9˃': 13471,
  '9': 43911,
  'H˃': 6737,
  '˂B': 18783,
  'T˃': 24487,
  '2˃': 20665,
  '2': 92980,
  '˂3': 5522,
  '3': 44779,
  '3˃': 16970,
  'K˃': 7186,
  '˂O': 8179,
  'Y˃': 19065,
  '˂V': 5222,
  'I˃': 11285,
  '8˃': 14023,
  '8': 32062,
  '˂2': 55589,
 

In [24]:
df.sort_values('q-error', ascending=False)

,query,true,est,q-error
406200,_6,2481,1.000000,2481.000000
83397,%R_AMIL%,4,472.969420,118.242355
159342,CLL%,1,110.870705,110.870705
42622,%E_ROLLEN_E%,1,79.914030,79.914032
44250,%FUIM%,1,71.012566,71.012566
...,...,...,...,...
311851,S TUKTAS KRLEKEN,1,0.862884,1.000000
311852,S T_BO%I B% TE%,1,0.831065,1.000000
311854,S WONDERFUL,1,0.995823,1.000000
311855,S _KKO_ A _ASA_,1,0.888833,1.000000


### Wildcards stat

In [4]:
df

,query,true,est,q-error,n_wild
0,% % % R%U%,474,653.777100,1.379277,5
1,% % % _E%,6346,7289.791504,1.148722,5
2,% % %A%A%O_R,24,132.570786,5.523783,6
3,% % %A_EL,218,337.574036,1.548505,4
4,% % %A_KADI,2,39.338467,19.669233,4
...,...,...,...,...,...
499995,_Z_U_ OZT_R_,1,6.026012,6.026012,5
499996,_Z_Y %G%,13,11.765779,1.104899,4
499997,_Z_Y _ Y_UNG,3,6.842568,2.280856,4
499998,_Z_ZA%A%RA_MA_,2,3.938846,1.969423,6


In [6]:

def count_wild(query):
    n_under = query.count('_')
    n_pcnt = query.count('%')
    return n_under + n_pcnt


data_names = ['DBLP-AN', 'IMDb-AN', 'IMDb-MT', 'TPCH-PN']
# data_names = ['IMDb-MT']
# data_names = ['DBLP-AN']
# data_names = ['IMDb-MT']
# models = ['EST_S', 'EST_M', 'EST_B', 'E2E', 'E2E-AUG', 'Astrid', 'Astrid-AUG', 'DREAM', 'DREAM-PACK', 'CLIQUE', 'CLIQUE-PACK', 'LPLM']
# models = ['DREAM', 'DREAM-PACK', 'CLIQUE', 'CLIQUE-PACK']
# models = ['LPLM','CLIQUE', 'CLIQUE-PACK', 'CLIQUE-PACK-ALL']
models = ['LPLM']
# workloads = ['CLIQUE', 'LPLM']
workloads = ['LPLM']


for workload in workloads:
    res_dict = {}
    for data_name in data_names:
        for model in models:
            # stat_path = os.path.join(res_dir, data_name, workload, 'stat', model, '0', 'stat.yml')
            stat_path = os.path.join(res_dir, data_name, workload, 'estimation', model, '0', 'estimation.csv')
            df = pd.read_csv(stat_path)
            df['n_wild'] = df['query'].apply(count_wild)
            print(data_name)
            print(f"{df['n_wild'].max() = }")
            print(f"{df['n_wild'].mean() = }")
            print(f"{df['true'].min() = }")
            print(f"{df['true'].max() = }")
            print(f"{df['true'].mean() = }")
            # print(f"{df['true_card'].median() = }")
            # df['len'] = df['query'].apply(len)
            # df['p'] = df['query'].apply(lambda x: 1)
            # df = df.groupby('n_wild').mean('q-error').transpose()
            # df = df.groupby('len').mean('q-error').transpose()
            # df = df.style.format(na_rep='', precision=2)
            # display(df)
            # display(df)

DBLP-AN
df['n_wild'].max() = 13
df['n_wild'].mean() = 3.256418
df['true'].min() = 0
df['true'].max() = 218670
df['true'].mean() = 143.584154
IMDb-AN
df['n_wild'].max() = 12
df['n_wild'].mean() = 3.025604
df['true'].min() = 1
df['true'].max() = 459738
df['true'].mean() = 157.561756
IMDb-MT
df['n_wild'].max() = 24
df['n_wild'].mean() = 4.047248
df['true'].min() = 1
df['true'].max() = 234970
df['true'].mean() = 215.980088
TPCH-PN
df['n_wild'].max() = 17
df['n_wild'].mean() = 6.324102
df['true'].min() = 1
df['true'].max() = 200000
df['true'].mean() = 497.292566


In [8]:
query_strings = ut.read_strings("data/title.title/query/CEB/train.txt")
df = pd.DataFrame(query_strings)
df['len'] = df[0].apply(len)
display(df)

,0,len
0,!Ne%,4
1,!Next?,6
2,!Que ve el%,11
3,"""A Corri%",9
4,"""Bookstore"" (%",14
...,...,...
3273742,à prendr%,9
3273743,ça p%,5
3273744,ìCàcaro!,8
3273745,ìMaten a ese%,13


In [9]:
display(df.sort_values('len'))

,0,len
3273746,ö,1
3271209,Å,1
2502710,D,1
2331484,4,1
2985272,R,1
...,...,...
423552,%Bass %in %Contra%,18
423554,%Bass %ro %ps AWA%,18
754007,%Shann%ve %t Dola%,18
1692538,%ney T% th%e Alps%,18


### Model size

In [35]:
model_size_dict = { # (Table_size, Model_size)
    "WIKI": (6484593, 3168885),
    "IMDB": (674953, 1708469),
    "DBLP": (618163, 1702453),
    "GENE": (157090, 1678837),
    "AUTHOR": (622099, 1698485),
    # "link_type.link": (7451, 1031733),
    "movie_companies.note": (206077, 840501),
    "name.name": (518383, 512437),
    "title.title": (607940, 420149),
}

In [36]:
res_size_dict = {x: (y[0]/1024/1024, y[1]/1024/1024, y[1]/sum(y)*100) for x, y in model_size_dict.items()}
df = pd.DataFrame(res_size_dict, index=['Table size(MB)', 'Model size(MB)', 'Model ratio(%)'])
pd.set_option('display.precision', 2)
display(df)
pd.reset_option('display.precision')

,WIKI,IMDB,DBLP,GENE,AUTHOR,movie_companies.note,name.name,title.title
Table size(MB),6.18,0.64,0.59,0.15,0.59,0.20,0.49,0.58
Model size(MB),3.02,1.63,1.62,1.60,1.62,0.80,0.49,0.40
Model ratio(%),32.83,71.68,73.36,91.44,73.19,80.31,49.71,40.87


In [14]:
from src.CLIQUE import is_LIKEquery_in_Table
ut.flip_last_canonicalized_like_query("%abc%__")
# is_LIKEquery_in_Table("G")

'%abc__%'

### TEST LPLM

In [32]:
from src.LPLM.extension import LIKE_pattern_to_extendedLanguage, LIKEpattern2mid_form
query = "Summ%vs._______"
# LIKE_pattern_to_extendedLanguage(query, True)
LIKEpattern2mid_form(query)

['˄', 'S', 'ˆ', 'u', 'ˆ', 'm', 'ˆ', 'm', 'v', 'ˆ', 's', 'ˆ', '.', '˕']